In [1]:
cd ..

C:\Users\Agustin\Dropbox\Doctorado\NeuroData\PyLeech


In [2]:
cd ..

C:\Users\Agustin\Dropbox\Doctorado\NeuroData


In [3]:
import PyLeech.Utils.CrawlingDatabaseUtils as CDU
import numpy as np
import matplotlib.pyplot as plt
import PyLeech.Utils.NLDUtils as NLD
import scipy.signal as spsig
import os.path
import winsound
from mpl_toolkits.mplot3d import Axes3D
import PyLeech.Utils.AbfExtension as abfe
import time
import gc
start = time.time()

In [4]:
%matplotlib qt5

In [5]:
cdd = CDU.loadDataDict()
file_list = list(cdd)
print(file_list)

['RegistrosDP_PP\\18n05010.pklspikes', 'RegistrosDP_PP\\2018_10_11_0002.pklspikes', 'RegistrosDP_PP\\2018_12_03_0005.pklspikes', 'RegistrosDP_PP\\2018_12_04_0003_0004_0005_0006b.pklspikes', 'RegistrosDP_PP\\2018_12_06_0005.pklspikes', 'RegistrosDP_PP\\2018_12_13_0015.pklspikes', 'RegistrosDP_PP\\NS_DP_PP_0.pklspikes', 'RegistrosDP_PP\\NS_T_DP_PP_0_cut.pklspikes', 'RegistrosDP_PP\\NS_T_DP_PP_1.pklspikes', 'RegistrosDP_PP\\2014_09_25_0008.pklspikes', 'RegistrosDP_PP\\2019_01_28_0001.pklspikes', 'RegistrosDP_PP\\2018_11_06_0004.pklspikes', 'RegistrosDP_PP\\2018_12_03_0000_0001.pklspikes', 'RegistrosDP_PP\\14217000.pklspikes', 'RegistrosDP_PP\\2017_12_08_0002.pklspikes', 'RegistrosDP_PP\\cont10.pklspikes', 'RegistrosDP_PP\\2018_12_13_0001.pklspikes', 'RegistrosDP_PP\\2019_07_22_0002.pklspikes', 'RegistrosDP_PP\\2019_07_22_0009.pklspikes', 'RegistrosDP_PP\\2019_07_22_0011.pklspikes', 'RegistrosDP_PP\\2019_07_23_0004.pklspikes', 'RegistrosDP_PP\\2019_07_23_0008.pklspikes', 'RegistrosDP_PP\\2

In [6]:
cdd = CDU.loadDataDict()
file_list = list(cdd)
binning_dt = .1
spike_kernel_sigma = 3

trace_list = []
emb_list = []
ran_files = []
run_dict = {}
file_sel = [
    '2019_07_22_0009', 
    '2019_07_22_0002',
    '2019_07_22_0011',
    '2019_08_30_0003',
    '2019_08_30_0006',
    '2018_12_03_0005',
    '2019_07_23_0008'
]
sublist = [fn for fn in file_list if any([fs in fn for fs in file_sel])]
print(sublist)
cycle_dict = {}

['RegistrosDP_PP\\2018_12_03_0005.pklspikes', 'RegistrosDP_PP\\2019_07_22_0002.pklspikes', 'RegistrosDP_PP\\2019_07_22_0009.pklspikes', 'RegistrosDP_PP\\2019_07_22_0011.pklspikes', 'RegistrosDP_PP\\2019_07_23_0008.pklspikes', 'RegistrosDP_PP\\2019_08_30_0003.pklspikes', 'RegistrosDP_PP\\2019_08_30_0006.pklspikes']


In [7]:
full_data = []
i = 0
for fn in sublist:
    ns_channel = [key for key, items in cdd[fn]['channels'].items() if 'NS' == items][0]
    fn = abfe.getAbFileListFromBasename(fn)[0]
    arr_dict, time_vector , fs = abfe.getArraysFromAbfFiles(fn, [ns_channel])
    NS_kernel = NLD.generateGaussianKernel(sigma=spike_kernel_sigma, time_range=20, dt_step=1 / fs)
    bl_kernel = NLD.generateGaussianKernel(sigma=45, time_range=10*60, dt_step=1 / fs)
    data = arr_dict[ns_channel] - np.mean(arr_dict[ns_channel])
    data[(data<-20) | (data>20)] = 0
    
    bl = spsig.fftconvolve(data, bl_kernel, mode='same')[::int(binning_dt * fs)]

    trace = spsig.fftconvolve(data, NS_kernel, mode='same')[::int(binning_dt * fs)]
    
#     fig, ax = plt.subplots()
#     ax.plot(trace)
#     ax.plot(trace-bl)
#     fig.suptitle(fn)
    
    if '2019_07_22_0009' in fn:
        trace = (trace-bl)[1600:9200]### solo porque estoy correndo el 2019 07 22 0009
#         peak_height = 2.4
    elif '2019_07_22_0002' in fn:
        trace = (trace-bl)[900:8800]### solo porque estoy correndo el 2019 07 22 0002
#         peak_height = 2.4
    elif '2019_07_22_0011' in fn:
        trace = (trace-bl)[4800:11000]### solo porque estoy correndo el 2019 07 22 0011
#         peak_height = 1
    elif '2019_08_30_0003' in fn:
        trace = (trace-bl)[2500:9600]### solo porque estoy correndo el 2019 07 22 0011
#         peak_height = 1
    elif '2019_08_30_0006' in fn:
        trace = (trace-bl)[1800:6800]
#         peak_height = 3.3
    elif '2019_07_23_0008' in fn:
            trace = (trace-bl)[3700:8400]
    elif '2018_12_03_0005' in fn:
            trace = (trace-bl)[2200:7800]
    else:
        trace = (trace-bl)
        peak_height = 1.5
            
    full_data.append(trace)

full_data = np.hstack(full_data).astype(np.float32)

In [8]:
fig, ax = plt.subplots()
ax.plot(full_data)
# every_single_cycle.shape

In [9]:
full_data.shape

(44100,)

In [10]:
step = 2
thr = .05
gc.collect()
cr_mat = NLD.getCloseReturns(full_data[::step, np.newaxis], threshold=thr, low_memory=True, get_mask=False)
cr_mat.shape

(22050, 22050)

In [11]:
cr_mat.shape[0] * cr_mat.shape[1] * int(str(cr_mat.dtype)[-2:]) / (8 * (1024 ** 3))

1.8112454563379288

In [12]:
NLD.plotCloseReturns(cr_mat[::5], masked=True, reorder=False, get_counts=False, thr=thr)

(<Figure size 640x480 with 2 Axes>,
 <matplotlib.axes._subplots.AxesSubplot at 0x2bd80621e48>)

In [12]:
gc.collect()
plt.matshow(cr_mat<(cr_mat.max()*.05))

In [13]:
gc.collect()
sg_dict = NLD.getCloseReturnsSegmentsFromUnorderedMatrix(cr_mat<(cr_mat.max()*.05), rt_len=100,single_file=True, min_dist=100)


In [14]:
time_delta = 100
length = 200
count = 0

for diagonal in list(sg_dict):
    for tup0, tup1 in sg_dict[diagonal]:
        if tup1 - tup0  > length and np.abs(diagonal) > time_delta:
            count += 1
print(count)

193


In [16]:
gc.collect()

0

In [17]:
# tup0, tup1 = 6575, 6775
# diag = 120
# fig1, ax1 = plt.subplots()
# t0 = every_single_cycle[::step][tup0:tup1]
# t1 = every_single_cycle[::step][tup0 + diagonal:tup1 + diagonal]
# r0 = np.arange(t0.shape[0])
# r1 = np.arange(t1.shape[0])
# ax1.plot(r0, t0, linewidth=1, color='k')
# ax1.plot(r1, t1, linewidth=1, color='r')

# fig1.suptitle(" shift: " + str(diagonal) + " " + str(tup0) + "-" + str(tup1) +
#              "\n" + str(tup0 + diagonal) + "-" + str(tup1 + diagonal))

In [18]:
# for diagonal in list(sg_dict):
#     for tup0, tup1 in sg_dict[diagonal]:
#         if tup1-tup0 > length and np.abs(diagonal)>time_delta:
#             fig1, ax1 = plt.subplots()
#             t0 = full_data[::step][tup0:tup1]
#             t1 = full_data[::step][tup0 + diagonal:tup1 + diagonal]
#             r0 = np.arange(t0.shape[0])
#             r1 = np.arange(t1.shape[0])
#             ax1.plot(r0, t0, linewidth=1, color='k')
#             ax1.plot(r1, t1, linewidth=1, color='r')

#             fig1.suptitle(" shift: " + str(diagonal) + " " + str(tup0) + "-" + str(tup1) +
#                          "\n" + str(tup0 + diagonal) + "-" + str(tup1 + diagonal))
            
            

In [15]:
import matplotlib as mpl
select = { 2067: (9817, 10041),
           2183: (10164, 10310),
           257: (13215, 13334),
#            213: (12312, 12561),
#            342: (814, 1097),
           588: (1215, 1435),
           190: (1220, 1435),
           185: (4804, 4960),
           176: (1055, 1275),
          
          
}

cmap = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=0, vmax=len(select)- 1)
scalarMap = mpl.cm.ScalarMappable(norm=cNorm, cmap=cmap)
ls = np.linspace(0, 1, len(select))


i = 0
fig = plt.figure()
ax = Axes3D(fig)
for diag, items in select.items():

    mean = np.mean(np.array((full_data[::step][items[0]:items[1]], full_data[::step][items[0]+diag:items[1]+diag])), axis=0)
    f1, a1 = plt.subplots()
    a1.plot(mean)
    f1.suptitle("diag " + str(diag) + ": " + str(items[0]) + "-" + str(items[1]))
#     emb = NLD.getTraceEmbedding(mean, 15, 3)
    emb = NLD.getDerivativeEmbedding(mean, .1, 3)
    NLD.plot3Dline(emb, fig_ax_pair=[fig, ax], label=str(diag), color=cmap(ls[i]))
    f1, a1 = NLD.plot3Dline(emb, label=str(diag), color=cmap(ls[i]))
    i += 1
    fig.suptitle("diag " + str(diag) + ": " + str(items[0]) + "-" + str(items[1]))
    f1.suptitle("diag " + str(diag) + ": " + str(items[0]) + "-" + str(items[1]))
    
    fig.legend()

In [32]:
i = 0 
diag, items =  176, (1055, 1275)
fig = plt.figure()
ax = Axes3D(fig)
mean = np.mean(np.array((full_data[::step][items[0]:items[1]], full_data[::step][items[0]+diag:items[1]+diag])), axis=0)
f1, a1 = plt.subplots()
a1.plot(mean, color='k')
a1.plot(full_data[::step][items[0]:items[1]])
a1.plot(full_data[::step][items[0]+diag:items[1]+diag])
f1.suptitle("diag " + str(diag) + ": " + str(items[0]) + "-" + str(items[1]))
# emb = NLD.getTraceEmbedding(mean, 15, 3)
emb = NLD.getDerivativeEmbedding(mean, .1, 3)
NLD.plot3Dline(emb, fig_ax_pair=[fig, ax], label=str(diag), color=cmap(ls[i]))

fig.suptitle("diag " + str(diag) + ": " + str(items[0]) + "-" + str(items[1]))

fig.legend()

c:\users\agus\appdata\local\programs\python\python36\lib\site-packages\findiff\findiff.py:242: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  yd[ref_multi_slice] += w * y[off_multi_slice]
c:\users\agus\appdata\local\programs\python\python36\lib\site-packages\findiff\findiff.py:240: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  yd[ref_multi_slice] += y[off_multi_slice]


In [31]:
emb = NLD.getTraceEmbedding(full_data, 15, 3)
fig, ax = NLD.plot3Dscatter(emb)